In [1]:
import re
import itertools

from sunpy.net import Fido, attrs as a
from sunpy.net.attr import SimpleAttr
from sunpy.net.dataretriever import GenericClient, QueryResponse
from sunpy.net.scraper import Scraper
from sunpy.time import TimeRange

In [2]:
import sunpy
sunpy.__version__

'7.0.0'

In [3]:
class DataType(SimpleAttr):
    """
    Attribute for specifying the data type for the search.

    Attributes
    ----------
    value : str
        The data type value.
    """


class PADREClient(GenericClient):
    """
    PADRE data client for accessing spectrum data from the PADRE mission.
    """

    pattern = (
        "https://umbra.nascom.nasa.gov/padre/padre-{{Instrument}}/{{Level}}/{{DataType}}/{{year:4d}}/{{month:2d}}/{{day:2d}}/"
        "padre_{{Instrument}}_{{Level}}_{{DataType}}_{{year:4d}}{{month:2d}}{{day:2d}}T{{hour:2d}}{{minute:2d}}{{second:2d}}_v0.1.0.fits"
    )

    @classmethod
    def register_values(cls):
        adict = {
            a.Provider: [("sdac", "The Solar Data Analysis Center.")],
            a.Source: [
                ("padre", "(The Solar Polarization and Directivity X-Ray Experiment)")
            ],
            a.Instrument: [
                (
                    "meddea",
                    "Measuring Directivity to Determine Electron Anisotropy (MeDDEA)",
                ),
                ("sharp", "Solar Hard X-ray Polarimeter (SHARP)"),
            ],
            DataType: [
                ("spectrum", "Spectrum data from MeDDEA."),
                ("photon", "Photon data from MeDDEA."),
                ("housekeeping", "Housekeeping data from MeDDEA."),
                ("unknown", "All the SHARP data has 'unknown' data type."),
            ],
            a.Level: [
                ("l0", "Raw data, converted to FITS, not in physical units."),
                ("l1", "Processed data, not in physical units."),
            ],
        }
        return adict

    def search(self, *args, **kwargs):
        """
        Query this client for a list of results.

        Parameters
        ----------
        \\*args: `tuple`
            `sunpy.net.attrs` objects representing the query.
        \\*\\*kwargs: `dict`
             Any extra keywords to refine the search.

        Returns
        -------
        A `QueryResponse` instance containing the query result.
        """
        # baseurl added for backwards compatibility purposes only
        _, pattern, matchdict = self.pre_search_hook(*args, **kwargs)
        # print(
        #     f"Finished _pre_search_hook with pattern: {pattern}, matchdict: {matchdict}"
        # )
        pattern_stem = "/".join(pattern.split("/")[:-1]) + "/"
        filename_pattern = pattern.split("/")[-1]

        scraper_urls = self.generate_url_permutations(pattern_stem, matchdict)
        # print(f"Generated URLs: {scraper_urls}")

        metalist = []
        for url_stem in scraper_urls:
            # Populated matchdict parameters into the URL stem & append filename pattern
            url_file_pattern = url_stem + filename_pattern
            # Create Scraper instance with the complete URL pattern
            scraper = Scraper(format=url_file_pattern)
            tr = TimeRange(matchdict["Start Time"], matchdict["End Time"])
            # Get File Metadata for the populated URL Pattern & Time Range
            filesmeta = scraper._extract_files_meta(tr, matcher=matchdict)
            # print(f"Finished _extract_files_meta with filesmeta: {filesmeta}")
            filesmeta = sorted(filesmeta, key=lambda k: k["url"])
            for i in filesmeta:
                rowdict = self.post_search_hook(i, matchdict)
                if rowdict:
                    metalist.append(rowdict)
        return QueryResponse(metalist, client=self)

    def generate_url_permutations(self, pattern, matchdict):
        # Extract placeholders from the pattern
        placeholder_regex = r"{{([^:}]+)(?::.*?)?}}"
        placeholders = re.findall(placeholder_regex, pattern)

        # Filter to only placeholders we have in our dictionary
        valid_placeholders = [p for p in placeholders if p in matchdict]

        # Get all values for each placeholder
        placeholder_values = [
            matchdict[p] if isinstance(matchdict[p], list) else [matchdict[p]]
            for p in valid_placeholders
        ]

        # Generate all combinations
        urls = []
        for combo in itertools.product(*placeholder_values):
            url = pattern
            for placeholder, value in zip(valid_placeholders, combo):
                url = url.replace(f"{{{{{placeholder}}}}}", str(value))
                # Handle any format specifiers
                format_pattern = re.compile(f"{{{{{placeholder}:.*?}}}}")
                url = format_pattern.sub(str(value), url)
            urls.append(url)

        return urls

In [4]:
# Show Added Client to Fido
Fido

Client,Description
CDAWEBClient,Provides access to query and download from the Coordinated Data Analysis Web (CDAWeb).
ADAPTClient,Provides access to the ADvanced Adaptive Prediction Technique (ADAPT) products of the National Solar Observatory (NSO).
AIASynopsisClient,"A client for retrieving AIA ""synoptic"" data from the JSOC."
EVEClient,Provides access to Level 0CS Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to several GOES XRS files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
GONGClient,Provides access to the Magnetogram products of NSO-GONG synoptic Maps.
LYRAClient,Provides access to the LYRA/Proba2 data archive.
NOAAIndicesClient,Provides access to the NOAA solar cycle indices.


In [5]:
results = Fido.search(
    a.Time("2025-05-01", "2025-05-05") & a.Instrument.meddea & a.Level.l1
)

In [6]:
import tempfile

with tempfile.TemporaryDirectory() as temp_dir:
    downloaded_files = Fido.fetch(results, path=temp_dir)
downloaded_files

Files Downloaded:   0%|          | 0/2 [00:00<?, ?file/s]

padre_meddea_l1_spectrum_20250504T000000_v0.1.0.fits:   0%|          | 0.00/8.26M [00:00<?, ?B/s]

padre_meddea_l1_housekeeping_20250504T000000_v0.1.0.fits:   0%|          | 0.00/150k [00:00<?, ?B/s]

['/var/folders/5l/_5r0pdg15fxg1_rkgmd3c1dm0000gn/T/tmpfx4i5fsi/padre_meddea_l1_spectrum_20250504T000000_v0.1.0.fits', '/var/folders/5l/_5r0pdg15fxg1_rkgmd3c1dm0000gn/T/tmpfx4i5fsi/padre_meddea_l1_housekeeping_20250504T000000_v0.1.0.fits']